<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Assignment: Notebook for Peer Assignment</font></h1>

# Introduction

Using this Python notebook you will:
1. Understand 3 Chicago datasets  
1. Load the 3 datasets into 3 tables in a Db2 database
1. Execute SQL queries to answer assignment questions 

## Understand the datasets 
To complete the assignment problems in this notebook you will be using three datasets that are available on the city of Chicago's Data Portal:
1. <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2">Socioeconomic Indicators in Chicago</a>
1. <a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t">Chicago Public Schools</a>
1. <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2">Chicago Crime Data</a>

### 1. Socioeconomic Indicators in Chicago
This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2



### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t




### 3. Chicago Crime Data 

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. 

This dataset is quite large - over 1.5GB in size with over 6.5 million rows. For the purposes of this assignment we will use a much smaller sample of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2


### Download the datasets
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the links below to download and save the datasets (.CSV files):
1. __CENSUS_DATA:__ https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv
1. __CHICAGO_PUBLIC_SCHOOLS__  https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv
1. __CHICAGO_CRIME_DATA:__ https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

__NOTE:__ Ensure you have downloaded the datasets using the links above instead of directly from the Chicago Data Portal. The versions linked here are subsets of the original datasets and have some of the column names modified to be more database friendly which will make it easier to complete this assignment.

### Store the datasets in database tables
To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in Week 3 Lab 3, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, __it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II__. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

<img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg">

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the first dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new tables as folows:
1. __CENSUS_DATA__
1. __CHICAGO_PUBLIC_SCHOOLS__
1. __CHICAGO_CRIME_DATA__

### Connect to the database 
Let us first load the SQL extension and establish a connection with the database

In [1]:
%reload_ext sql

In the next cell enter your db2 connection string. Recall you created Service Credentials for your Db2 instance in first lab in Week 3. From the __uri__ field of your Db2 service credentials copy everything after db2:// (except the double quote at the end) and paste it in the cell below after ibm_db_sa://

<img src ="https://ibm.box.com/shared/static/hzhkvdyinpupm2wfx49lkr71q9swbpec.jpg">

In [2]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
# Enter the connection string for your Db2 on Cloud database instance below
%sql ibm_db_sa://fmw28835:k6vr9%2Bvbglc5011q@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB

'Connected: fmw28835@BLUDB'

In [3]:
import ibm_db
import pandas
import ibm_db_dbi

In [7]:
pconn = ibm_db_dbi.Connection(conn)

In [8]:
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net"            # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "50000"                    # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "fmw28835"                 # e.g. "abc12345"
dsn_pwd = "k6vr9+vbglc5011q"    

In [9]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  fmw28835 on host:  dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net


## Problems
Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table

In [10]:
# Rows in Crime table
#Construct the query that retrieves all rows from the CRIME table
selectQuery = "SELECT * from CRIME_DATA"

#Execute the statement
crime_df = pandas.read_sql(selectQuery, pconn)
len(crime_df)

533

In [51]:
%%sql
SELECT COUNT(*) FROM CRIME_DATA

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


1
533


### Problem 2

##### Retrieve first 10 rows from the CRIME table


In [52]:
%%sql
SELECT * FROM CRIME_DATA LIMIT 10

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Column_0,id,case_number,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,latitude,longitude,location
0,3512276,HK587712,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.807440500000006,-87.70395585000000,"(41.8074405, -87.703955849)"
1,3406613,HK456306,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.89827996,-87.71640551000000,"(41.898279962, -87.716405505)"
2,8002131,HT233595,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,221,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.81593313,-87.62464213000000,"(41.815933131, -87.624642127)"
3,7903289,HT133522,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,423,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.74366532,-87.56246276000000,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,831,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.773455299999995,-87.70648047000000,"(41.773455295, -87.706480471)"
5,7732712,HS540106,006XX W CHICAGO AVE,810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),False,False,1323,12,27.0,24.0,6,1171668.0,1905607.0,2010,41.89644677,-87.64493868000000,"(41.896446772, -87.644938678)"
6,10769475,HZ534771,050XX N KEDZIE AVE,810,THEFT,OVER $500,STREET,False,False,1713,17,33.0,14.0,6,1154133.0,1933314.0,2016,41.97284491,-87.70860008000000,"(41.972844913, -87.708600079)"
7,4494340,HL793243,005XX E PERSHING RD,860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,True,False,213,2,3.0,38.0,6,1180448.0,1879234.0,2005,41.82387989,-87.61350386000000,"(41.823879885, -87.613503857)"
8,3778925,HL149610,100XX S WASHTENAW AVE,810,THEFT,OVER $500,STREET,False,False,2211,22,19.0,72.0,6,1160129.0,1838040.0,2005,41.71128051,-87.68917909999999,"(41.711280513, -87.689179097)"
9,3324217,HK361551,033XX W BELMONT AVE,820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,False,False,1733,17,35.0,21.0,6,1153590.0,1921084.0,2004,41.93929582,-87.71092344000000,"(41.939295821, -87.710923442)"


In [11]:
selectQuery = "SELECT * from CRIME_DATA LIMIT 10"
crime_10_df = pandas.read_sql(selectQuery, pconn)
crime_10_df

,Column_0,ID,CASE_NUMBER,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,0,3512276,HK587712,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.807440500000006,-87.703956,"(41.8074405, -87.703955849)"
1,1,3406613,HK456306,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.89827996,-87.716406,"(41.898279962, -87.716405505)"
2,2,8002131,HT233595,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.81593313,-87.624642,"(41.815933131, -87.624642127)"
3,3,7903289,HT133522,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.74366532,-87.562463,"(41.743665322, -87.562462756)"
4,4,10402076,HZ138551,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.773455299999995,-87.706480,"(41.773455295, -87.706480471)"
5,5,7732712,HS540106,006XX W CHICAGO AVE,810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),False,False,...,12,27.0,24.0,6,1171668.0,1905607.0,2010,41.89644677,-87.644939,"(41.896446772, -87.644938678)"
6,6,10769475,HZ534771,050XX N KEDZIE AVE,810,THEFT,OVER $500,STREET,False,False,...,17,33.0,14.0,6,1154133.0,1933314.0,2016,41.97284491,-87.708600,"(41.972844913, -87.708600079)"
7,7,4494340,HL793243,005XX E PERSHING RD,860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,True,False,...,2,3.0,38.0,6,1180448.0,1879234.0,2005,41.82387989,-87.613504,"(41.823879885, -87.613503857)"
8,8,3778925,HL149610,100XX S WASHTENAW AVE,810,THEFT,OVER $500,STREET,False,False,...,22,19.0,72.0,6,1160129.0,1838040.0,2005,41.71128051,-87.689179,"(41.711280513, -87.689179097)"
9,9,3324217,HK361551,033XX W BELMONT AVE,820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,False,False,...,17,35.0,21.0,6,1153590.0,1921084.0,2004,41.93929582,-87.710923,"(41.939295821, -87.710923442)"


### Problem 3

##### How many crimes involve an arrest?

In [54]:
%%sql
SELECT COUNT(*) FROM CRIME_DATA WHERE ARREST='True'

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


1
163


In [12]:
selectQuery = "SELECT * from CRIME_DATA WHERE ARREST='True'"
arrest_df = pandas.read_sql(selectQuery, pconn)
len(arrest_df)

163

### Problem 4

##### Which unique types of crimes have been recorded at GAS STATION locations?


In [55]:
%%sql
SELECT DISTINCT PRIMARY_TYPE FROM CRIME_DATA WHERE LOCATION_DESCRIPTION = 'GAS STATION'

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


primary_type
CRIMINAL TRESPASS
NARCOTICS
ROBBERY
THEFT


In [13]:
selectQuery = "SELECT DISTINCT PRIMARY_TYPE FROM CRIME_DATA WHERE LOCATION_DESCRIPTION = 'GAS STATION'"
crime_type_df = pandas.read_sql(selectQuery, pconn)
crime_type_df

,PRIMARY_TYPE
0,CRIMINAL TRESPASS
1,NARCOTICS
2,ROBBERY
3,THEFT


Hint: Which column lists types of crimes e.g. THEFT?

### Problem 5

##### In the CENUS_DATA table list all Community Areas whose names start with the letter ‘B’.

In [56]:
%%sql
SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NAME LIKE 'B%'

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name
Belmont Cragin
Burnside
Brighton Park
Bridgeport
Beverly


In [14]:
selectQuery = "SELECT COMMUNITY_AREA_NAME from CENSUS_DATA WHERE COMMUNITY_AREA_NAME LIKE 'B%'"

#Execute the statement
census_comm_df = pandas.read_sql(selectQuery, pconn)
census_comm_df

,COMMUNITY_AREA_NAME
0,Belmont Cragin
1,Burnside
2,Brighton Park
3,Bridgeport
4,Beverly


### Problem 6

##### Which schools in Community Areas 10 to 15 are healthy school certified?

In [58]:
%%sql
SELECT NAME_OF_SCHOOL FROM CHICAGO_PUBLIC_SCHOOLS WHERE COMMUNITY_AREA_NUMBER > '9' AND COMMUNITY_AREA_NUMBER <'16' AND HEALTHY_SCHOOL_CERTIFIED = 'Yes'

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


name_of_school
Rufus M Hitch Elementary School


In [15]:
selectQuery = "SELECT * FROM CHICAGO_PUBLIC_SCHOOLS"

#Execute the statement
schools_df = pandas.read_sql(selectQuery, pconn)
for col in schools_df.columns:
    print(col)

School_ID
NAME_OF_SCHOOL
Elementary, Middle, or High School
Street_Address
City
State
ZIP_Code
Phone_Number
Link
Network_Manager
Collaborative_Name
Adequate_Yearly_Progress_Made_
Track_Schedule
CPS_Performance_Policy_Status
CPS_Performance_Policy_Level
HEALTHY_SCHOOL_CERTIFIED
Safety_Icon
SAFETY_SCORE
Family_Involvement_Icon
Family_Involvement_Score
Environment_Icon
Environment_Score
Instruction_Icon
Instruction_Score
Leaders_Icon
Leaders_Score
Teachers_Icon
Teachers_Score
Parent_Engagement_Icon
Parent_Engagement_Score
Parent_Environment_Icon
Parent_Environment_Score
AVERAGE_STUDENT_ATTENDANCE
Rate_of_Misconducts__per_100_students_
Average_Teacher_Attendance
Individualized_Education_Program_Compliance_Rate
Pk_2_Literacy__
Pk_2_Math__
Gr3_5_Grade_Level_Math__
Gr3_5_Grade_Level_Read__
Gr3_5_Keep_Pace_Read__
Gr3_5_Keep_Pace_Math__
Gr6_8_Grade_Level_Math__
Gr6_8_Grade_Level_Read__
Gr6_8_Keep_Pace_Math_
Gr6_8_Keep_Pace_Read__
Gr_8_Explore_Math__
Gr_8_Explore_Read__
ISAT_Exceeding_Math__
ISA

In [16]:
schools_df['College_Enrollment_Rate__'].unique()

array(['NDA', '79.8', '64.2', '51.3', '48.8', '73.2', '40.4', '58', '47',
       '79.6', '69.3', '54', '36.7', '31.3', '63.3', '85.1', '51', '37',
       '40.9', '43.1', '45.1', '39.5', '78.5', '40.3', '44', '60.5',
       '50.9', '57.4', '87.9', '68.8', '58.2', '57.6', '63.5', '55.7',
       '52.9', '45.9', '45.2', '35', '79.7', '62.8', '75.5', '51.9',
       '61.3', '60.4', '58.5', '74', '52.4', '54.4', '41.3', '90.7',
       '54.9', '51.5', '42.2', '32.6', '49.1', '85.9', '41.5', '52.1',
       '65.2', '42.9', '46.4', '65', '82.4', '43.8', '42.6', '88.3',
       '62.2', '50', '86.9', '48.1'], dtype=object)

In [17]:
schools_df[(schools_df['COMMUNITY_AREA_NUMBER']>9) & (schools_df['COMMUNITY_AREA_NUMBER']<16) & (schools_df['HEALTHY_SCHOOL_CERTIFIED']=='Yes')]

,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,...,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
471,609995,Rufus M Hitch Elementary School,ES,5625 N McVicker Ave,Chicago,IL,60646,(773) 534-1189,http://schoolreports.cps.edu/SchoolProgressRep...,O'Hare Elementary Network,...,NDA,1134940.195,1937129.571,41.983678,-87.779086,10,NORWOOD PARK,45,16,"(41.98367756, -87.77908614)"


In [18]:
selectQuery = "SELECT * FROM CHICAGO_PUBLIC_SCHOOLS WHERE COMMUNITY_AREA_NUMBER > '9' AND COMMUNITY_AREA_NUMBER <'16' AND HEALTHY_SCHOOL_CERTIFIED = 'Yes' "

#Execute the statement
schools_1015_health_certified_df = pandas.read_sql(selectQuery, pconn)
schools_1015_health_certified_df

,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,...,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
0,609995,Rufus M Hitch Elementary School,ES,5625 N McVicker Ave,Chicago,IL,60646,(773) 534-1189,http://schoolreports.cps.edu/SchoolProgressRep...,O'Hare Elementary Network,...,NDA,1134940.195,1937129.571,41.983678,-87.779086,10,NORWOOD PARK,45,16,"(41.98367756, -87.77908614)"


### Problem 7

##### What is the average school Safety Score? 

In [59]:
%%sql
SELECT AVG(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


1
49.504873


In [19]:
selectQuery = "SELECT AVG(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS"

avg_safety_score_df = pandas.read_sql(selectQuery, pconn)
avg_safety_score_df

,1
0,49.504873


### Problem 8

##### List the top 5 Community Areas by average College Enrollment [number of students] 

In [68]:
%%sql
SELECT COMMUNITY_AREA_NAME, AVG(COLLEGE_ENROLLMENT) as number_of_students 
FROM CHICAGO_PUBLIC_SCHOOLS 
GROUP BY COMMUNITY_AREA_NAME 
ORDER BY number_of_students 
DESC LIMIT 5

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name,number_of_students
ARCHER HEIGHTS,2411.500000
MONTCLARE,1317.000000
WEST ELSDON,1233.333333
BRIGHTON PARK,1205.875000
BELMONT CRAGIN,1198.833333


In [65]:
%%sql
SELECT COLLEGE_ENROLLMENT_RATE__ FROM CHICAGO_PUBLIC_SCHOOLS

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  "COLLEGE_ENROLLMENT_RATE__" is not valid in the context where it is used.  SQLSTATE=42703 SQLCODE=-206
[SQL: SELECT COLLEGE_ENROLLMENT_RATE__ FROM CHICAGO_PUBLIC_SCHOOLS]
(Background on this error at: http://sqlalche.me/e/13/f405)


In [63]:
%%sql
SELECT COMMUNITY_AREA_NAME, COLLEGE_ENROLLMENT FROM CHICAGO_PUBLIC_SCHOOLS ORDER BY COLLEGE_ENROLLMENT DESC LIMIT 5

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name,college_enrollment
NORTH CENTER,4368
ARCHER HEIGHTS,3320
NORWOOD PARK,2922
BRIGHTON PARK,2883
IRVING PARK,2366


In [35]:
selectQuery = "SELECT COMMUNITY_AREA_NAME FROM CHICAGO_PUBLIC_SCHOOLS ORDER BY COLLEGE_ENROLLMENT_RATE__ DESC LIMIT 5"
top_community_area_by_coll_enroll_df = pandas.read_sql(selectQuery, pconn)
top_community_area_by_coll_enroll_df

DatabaseError: Execution failed on sql 'SELECT COMMUNITY_AREA_NAME FROM CHICAGO_PUBLIC_SCHOOLS ORDER BY COLLEGE_ENROLLMENT_RATE': ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  "COLLEGE_ENROLLMENT_RATE" is not valid in the context where it is used.  SQLSTATE=42703 SQLCODE=-206

### Problem 9

##### Use a sub-query to determine which Community Area has the least value for school Safety Score? 

In [66]:
%%sql
SELECT COMMUNITY_AREA_NAME, SAFETY_SCORE FROM CHICAGO_PUBLIC_SCHOOLS WHERE SAFETY_SCORE = (SELECT MIN(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS)

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name,safety_score
WASHINGTON PARK,1


In [44]:
selectQuery = "SELECT COMMUNITY_AREA_NAME,SAFETY_SCORE FROM CHICAGO_PUBLIC_SCHOOLS WHERE SAFETY_SCORE = (SELECT MIN(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS)"
safety_df = pandas.read_sql(selectQuery, pconn)
safety_df

,COMMUNITY_AREA_NAME,SAFETY_SCORE
0,WASHINGTON PARK,1


### Problem 10

##### [Without using an explicit JOIN operator] Find the Per Capita Income of the Community Area which has a school Safety Score of 1.

In [70]:
%%sql
SELECT PER_CAPITA_INCOME, COMMUNITY_AREA_NAME 
FROM CENSUS_DATA 
WHERE COMMUNITY_AREA_NUMBER = 
(SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_PUBLIC_SCHOOLS WHERE SAFETY_SCORE = 1)

 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


per_capita_income,community_area_name
13785,Washington Park


In [46]:
#From problem 9 I Know Washington Park has a safety score of 1.
selectQuery = "SELECT PER_CAPITA_INCOME,COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NAME LIKE 'Washington Park'"
pci_df = pandas.read_sql(selectQuery, pconn)
pci_df

,PER_CAPITA_INCOME,COMMUNITY_AREA_NAME
0,13785,Washington Park


 * ibm_db_sa://fmw28835:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  "COLLEGE_ENROLLMENT_RATE" is not valid in the context where it is used.  SQLSTATE=42703 SQLCODE=-206
[SQL: SELECT COLLEGE_ENROLLMENT_RATE FROM CHICAGO_PUBLIC_SCHOOLS]
(Background on this error at: http://sqlalche.me/e/13/f405)


Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
